<a href="https://colab.research.google.com/github/Caleb-Flegel/CS2---Code-Example/blob/master/Two_Slit_Bright_Light.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Partner: Zach Washburn

Date: 9/20/2022

# Objectives
* To measure both of the slit widths in the double slit slide using
their respective single-slit diffraction patterns.
* To determine the laser wavelength using by fitting the Fraunhofer
model for two-slit interference to the observed intensity pattern.


# Install and import libraries

In [1]:
!pip install whit-phys-util 1>/dev/null

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import whit_phys_util as buc

Mounted at /content/drive


# Connect to GitHub

In [3]:
repo = buc.local_repository("https://github.com/PS251-FA22/two-slit-bright-1b")

Cloning repository from GitHub.


# Equipment

| Description | Manufacturer | Model | ID or SN |
| ----------- | ------------ | ----- | -------- |
| Two-slit interference, one-photon-at-a-time apparatus | TeachSpin | TWS 2-A | TWS 383 |
| multimeter | Fluke | 179 | 58870154 |


# Slit-blocker positions
* Far slit unblocked: 4.35 mm
* Both unblocked: 5.65 mm
* Near slit unblocked: 6.09 mm

In [ ]:
distanceToDetector = 19100 # distance from double-slit to detector in mm

19100 mm

# Key landmarks in the interference pattern

| Label | Location (mm) | Photodiode Voltage (V) | 
| ----- | ------------- | ---------------------- |
|$\mathrm{3_f}$ | 3.41 | 0.600 |
|$\mathrm{2_f}$ | 4.21 | 1.549 |
|$\mathrm{1_f}$ | 5.15 | 2.498 |
|$0$            | 5.81 | 2.809 | 
|$\mathrm{1_n}$ | 6.72 | 2.498 |
|$\mathrm{2_n}$ | 7.50 | 1.521 |
|$\mathrm{3_n}$ | 8.26 | 0.588 |

We encountered good differences in volts and location between the three peaks we measured in either direction of the maximum. One possible issues is that our peak voltage is lower than it ideally should be. 